In [ ]:
import tensorflow as tf
import keras
from keras import backend as K
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
print(tf.__version__)
print(keras.__version__)
import cv2  # for image processing
from sklearn.model_selection import train_test_split
import scipy.io
import os
import h5py
from arts_preprocess_utils import load_dataset
from keras.preprocessing.image import ImageDataGenerator
from scipy.spatial.distance import directed_hausdorff, cosine

In [ ]:
# !!! remember to clear session/graph if you rebuild your graph to avoid out-of-memory errors !!!
def reset_tf_session():
    K.clear_session()
    tf.reset_default_graph()
    s = K.get_session()
    return s

In [ ]:
reset_tf_session()

## Load dataset

In [ ]:
train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig = load_dataset('./wikiart_mini_ic.h5')

In [ ]:
img_Height = train_set_x_orig.shape[1]
img_Width = train_set_x_orig.shape[2]
N_CLASSES = len(np.unique(test_set_y_orig))-1

In [ ]:
X_train = train_set_x_orig * (1./255)
y_train = train_set_y_orig * (1./255)

X_dev = test_set_x_orig * (1./255)
y_dev = test_set_y_orig * (1./255)

**Plot image**

In [ ]:
plt.imshow(X_dev[7][...,::-1])

## Autoencoder

In [ ]:
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense, BatchNormalization, GlobalAveragePooling2D, Reshape, LeakyReLU, Dropout, UpSampling2D, Conv2DTranspose

def build_deep_autoencoder(img_shape,code_size=300, filters = 40):
    """PCA's deeper brother. See instructions above"""
    
    #encoder architecture
    encoder = Sequential()
    
    #output shape: 128x128xfilters
    encoder.add(Conv2D(filters, kernel_size=(4,4), strides=(2,2), padding='same', input_shape=img_shape))
    encoder.add(BatchNormalization(momentum=0.8))
    encoder.add(LeakyReLU())

    #output shape: 64x64xfilters*2
    encoder.add(Conv2D(filters*2, kernel_size=(4,4), strides=(2,2), padding='same'))
    encoder.add(BatchNormalization(momentum=0.8))
    encoder.add(LeakyReLU())

    #output shape: 32x32xfilters*4
    encoder.add(Conv2D(filters*4, kernel_size=(4,4), strides=(2,2), padding='same'))
    encoder.add(BatchNormalization(momentum=0.8))
    encoder.add(LeakyReLU())

    #output shape: 16x16xfilters*8
    encoder.add(Conv2D(filters*8, kernel_size=(4,4), strides=(2,2), padding='same'))
    encoder.add(BatchNormalization(momentum=0.8))
    encoder.add(LeakyReLU())

    #output shape: 8x8xfilters*16
    encoder.add(Conv2D(filters*16, kernel_size=(4,4), strides=(2,2), padding='same'))
    encoder.add(BatchNormalization(momentum=0.8))
    encoder.add(LeakyReLU())

    #output shape: 4x4xfilters*32
    encoder.add(Conv2D(filters*32, kernel_size=(4,4), strides=(2,2), padding='same'))
    encoder.add(BatchNormalization(momentum=0.8))
    encoder.add(LeakyReLU())

    #output shape: 1x1xcode_size
    encoder.add(Conv2D(code_size, kernel_size=(4,4)))
    encoder.add(BatchNormalization(momentum=0.8))
    encoder.add(LeakyReLU())

    #decoder architecture
    decoder = Sequential()

    #output shape: 4x4xfilters*32
    decoder.add(Conv2DTranspose(filters * 32, kernel_size=(4, 4),input_shape=(1,1,code_size)))
    decoder.add(BatchNormalization())
    decoder.add(Activation(activation='relu'))

    #output shape: 8x8xfilters*16
    decoder.add(Conv2DTranspose(filters * 16, kernel_size=(4,4), strides=(2,2), padding='same'))
    decoder.add(BatchNormalization())
    decoder.add(Activation(activation='relu'))

    #output shape: 16x16xfilters*8
    decoder.add(Conv2DTranspose(filters * 8, kernel_size=(4,4), strides=(2,2), padding='same'))
    decoder.add(BatchNormalization())
    decoder.add(Activation(activation='relu'))

    #output shape: 32x32xfilters*4
    decoder.add(Conv2DTranspose(filters * 4, kernel_size=(4,4), strides=(2,2), padding='same'))
    decoder.add(BatchNormalization())
    decoder.add(Activation(activation='relu'))

    #output shape: 64x64xfilters*2
    decoder.add(Conv2DTranspose(filters * 2, kernel_size=(4,4), strides=(2,2), padding='same'))
    decoder.add(BatchNormalization())
    decoder.add(Activation(activation='relu'))

    #output shape: 128x128xfilters
    decoder.add(Conv2DTranspose(filters, kernel_size=(4,4), strides=(2,2), padding='same'))
    decoder.add(BatchNormalization())
    decoder.add(Activation(activation='relu'))

    #output shape: 256x256x3
    decoder.add(Conv2DTranspose(3, kernel_size=(4,4), strides=(2,2), padding='same'))
    decoder.add(BatchNormalization())
    decoder.add(Activation(activation='sigmoid'))
    
    return encoder,decoder

In [ ]:
encoder, decoder = build_deep_autoencoder(img_shape=(img_Height, img_Width, 3))

**Build autoencoder**

In [ ]:
inp = keras.layers.Input((img_Height,img_Width,3))
code = encoder(inp)
reconstruction = decoder(code)

autoencoder = Model(inputs=inp, outputs=reconstruction)

In [ ]:
autoencoder.summary()

In [ ]:
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

**Add noise and fit data**

In [ ]:
X_train_noisy = X_train + 0.5 * np.random.normal(loc=0.0, scale=1.0, size=X_train.shape)
X_dev_noisy = X_dev + 0.5 * np.random.normal(loc=0.0, scale=1.0, size=X_dev.shape)

In [ ]:
autoencoder.fit(x=X_train_noisy,y=X_train,
                epochs=128,
                batch_size=64,
                shuffle=True, 
                validation_data=(X_dev_noisy, X_dev))

In [ ]:
encoder.save_weights('encoder01.h5')

In [ ]:
decoder.save_weights('decoder01.h5')

**Test autoencoder**

In [ ]:
code = encoder.predict(X_train_noisy[0:10])

In [ ]:
code.shape

In [ ]:
plt.imshow(code.reshape([code[6].shape[-1]//2,-1]))

In [ ]:
reconstructed = decoder.predict(code)

In [ ]:
plt.imshow(reconstructed[4][...,::-1])

In [ ]:
plt.imshow(np.clip(X_train_noisy[4], 0, 1)[...,::-1])

In [ ]:
plt.imshow(X_train[4][...,::-1])

**Encode random image**

In [ ]:
img = plt.imread('./Bren.jpg')

In [ ]:
img = cv2.resize(img, (img_Height, img_Width), interpolation=cv2.INTER_CUBIC)
img_norm = img *(1./255)
img_norm = np.expand_dims(img_norm, axis=0)

In [ ]:
plt.imshow(img_norm[0])

In [ ]:
code = encoder.predict(img_norm)
reconstructed = decoder.predict(code)

In [ ]:
plt.imshow(reconstructed[0])

In [ ]:
reconstructed = decoder.predict(np.random.uniform(0, 1, size=[10, 1, 1, 300]))